<a href="https://colab.research.google.com/github/mtoce/DS-Unit-4-Sprint-2-Neural-Networks/blob/master/module2-backpropagation/LS_DS_422_Backprop_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Backpropagation Practice

## *Data Science Unit 4 Sprint 2 Assignment 2*

Using TensorFlow Keras, Implement a 3 input, 4 node hidden-layer, 1 output node Multilayer Perceptron on the following dataset:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 0  | 0  | 1  | 0 |
| 0  | 1  | 1  | 1 |
| 1  | 0  | 1  | 1 |
| 0  | 1  | 0  | 1 |
| 1  | 0  | 0  | 1 |
| 1  | 1  | 1  | 0 |
| 0  | 0  | 0  | 0 |

If you look at the data you'll notice that the first two columns behave like an XOR gate while the last column is mostly just noise. Remember that creating an XOR gate was what the perceptron was criticized for not being able to learn.

This is your "Hello World!" of TensorFlow.

### Example TensorFlow Starter Code

```python 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
    Dense(3, activation='sigmoid', input_dim=2),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['acc'])

results = model.fit(X,y, epochs=100)

```

### Additional Written Tasks:
1. Investigate the various [loss functions](https://www.tensorflow.org/api_docs/python/tf/keras/losses). Which is best suited for the task at hand (predicting 1 / 0) and why? 
2. What is the difference between a loss function and a metric? Why might we need both in Keras? 
3. Investigate the various [optimizers](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers). Stochastic Gradient Descent (`sgd`) is not the learning algorithm dejour anyone. Why is that? What do newer optimizers such as `adam` have to offer? 

In [0]:
import pandas as pd
import numpy as np

# create simple dict of data
data = {
    "x1": [0,0,1,0,1,1,0],
    "x2": [0,1,0,1,0,1,0],
    "x3": [1,1,1,0,0,1,0],
    "y": [0,1,1,1,1,0,0]
}

df = pd.DataFrame(data)
df.head()

,x1,x2,x3,y
0,0,0,1,0
1,0,1,1,1
2,1,0,1,1
3,0,1,0,1
4,1,0,0,1


In [0]:
X = df[["x1", "x2", "x3"]].values
y = df[["y"]].values
X = X / np.amax(X, axis=0)
y = y / 100

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
    Dense(3, activation='sigmoid', input_dim=3),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['acc'])



results = model.fit(X, y, epochs=100)

Epoch 1/100
1/1 [==============================] - 0s 2ms/step - loss: 0.2182 - acc: 0.4286
Epoch 2/100
1/1 [==============================] - 0s 2ms/step - loss: 0.2175 - acc: 0.4286
Epoch 3/100
1/1 [==============================] - 0s 2ms/step - loss: 0.2168 - acc: 0.4286
Epoch 4/100
1/1 [==============================] - 0s 2ms/step - loss: 0.2160 - acc: 0.4286
Epoch 5/100
1/1 [==============================] - 0s 2ms/step - loss: 0.2153 - acc: 0.4286
Epoch 6/100
1/1 [==============================] - 0s 2ms/step - loss: 0.2146 - acc: 0.4286
Epoch 7/100
1/1 [==============================] - 0s 2ms/step - loss: 0.2139 - acc: 0.4286
Epoch 8/100
1/1 [==============================] - 0s 2ms/step - loss: 0.2132 - acc: 0.4286
Epoch 9/100
1/1 [==============================] - 0s 2ms/step - loss: 0.2125 - acc: 0.4286
Epoch 10/100
1/1 [==============================] - 0s 2ms/step - loss: 0.2118 - acc: 0.4286
Epoch 11/100
1/1 [==============================] - 0s 2ms/step - loss: 0.2111 

In [0]:
scores = model.evaluate(X, y)
print(f"{model.metrics_names[1]}: {scores[1]*100}")

1/1 [==============================] - 0s 1ms/step - loss: 0.1632 - acc: 0.4286
acc: 42.85714328289032


### Build a Tensor Keras Perceptron

Try to match the architecture we used on Monday - inputs nodes and one output node. Apply this architecture to the XOR-ish dataset above. 

After fitting your model answer these questions: 

Are you able to achieve the same results as a bigger architecture from the first part of the assignment? Why is this disparity the case? What properties of the XOR dataset would cause this disparity? 

Now extrapolate this behavior on a much larger dataset in terms of features. What kind of architecture decisions could we make to avoid the problems the XOR dataset presents at scale? 

*Note:* The bias term is baked in by default in the Dense layer.

## Try building/training a more complex MLP on a bigger dataset.

Use TensorFlow Keras & the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to build the canonical handwriting digit recognizer and see what kind of accuracy you can achieve. 

If you need inspiration, the Internet is chalk-full of tutorials, but I want you to see how far you can get on your own first. I've linked to the original MNIST dataset above but it will probably be easier to download data through a neural network library. If you reference outside resources make sure you understand every line of code that you're using from other sources, and share with your fellow students helpful resources that you find.


### Parts
1. Gathering & Transforming the Data
2. Making MNIST a Binary Problem
3. Estimating your Neural Network (the part you focus on)

### Gathering the Data 

`keras` has a handy method to pull the mnist dataset for you. You'll notice that each observation is a 28x28 arrary which represents an image. Although most Neural Network frameworks can handle higher dimensional data, that is more overhead than necessary for us. We need to flatten the image to one long row which will be 784 values (28X28). Basically, you will be appending each row to one another to make on really long row. 

In [0]:
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [0]:
# input image dimensions
img_rows, img_cols = 28, 28

In [0]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [0]:
x_train = x_train.reshape(x_train.shape[0], img_rows * img_cols)
x_test = x_test.reshape(x_test.shape[0], img_rows * img_cols)

# Normalize Our Data
x_train = x_train / 255
x_test = x_test / 255

In [0]:
# Now the data should be in a format you're more familiar with
x_train.shape

(60000, 784)

### Making MNIST a Binary Problem 
MNIST is multiclass classification problem; however we haven't covered all the necessary techniques to handle this yet. You would need to one-hot encode the target, use a different loss metric, and use softmax activations for the last layer. This is all stuff we'll cover later this week, but let us simplify the problem for now: Zero or all else.

In [0]:
import numpy as np

y_temp = np.zeros(y_train.shape)
y_temp[np.where(y_train == 0.0)[0]] = 1
y_train = y_temp

y_temp = np.zeros(y_test.shape)
y_temp[np.where(y_test == 0.0)[0]] = 1
y_test = y_temp

In [0]:
# A Nice Binary target for ya to work with
y_train.shape

(60000,)

### Estimating Your `net

In [0]:
# Instantiating Model
model = Sequential([
    Dense(10, activation="sigmoid", input_dim=784),
    Dense(1, activation="sigmoid")
])
# Compile
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
# Results
results = model.fit(x_train, y_train, epochs=10)
# Score
scores = model.evaluate(x_test, y_test)
print(f"{model.metrics_names[1]}: {scores[1]*100}")

Epoch 1/10
1875/1875 [==============================] - 3s 1ms/step - loss: 1.1768e-08 - accuracy: 0.0987
Epoch 2/10
1875/1875 [==============================] - 3s 1ms/step - loss: 1.1768e-08 - accuracy: 0.0987
Epoch 3/10
1875/1875 [==============================] - 3s 1ms/step - loss: 1.1768e-08 - accuracy: 0.0987
Epoch 4/10
1875/1875 [==============================] - 3s 1ms/step - loss: 1.1768e-08 - accuracy: 0.0987
Epoch 5/10
1875/1875 [==============================] - 3s 1ms/step - loss: 1.1768e-08 - accuracy: 0.0987
Epoch 6/10
1875/1875 [==============================] - 3s 1ms/step - loss: 1.1768e-08 - accuracy: 0.0987
Epoch 7/10
1875/1875 [==============================] - 3s 2ms/step - loss: 1.1768e-08 - accuracy: 0.0987
Epoch 8/10
1875/1875 [==============================] - 3s 2ms/step - loss: 1.1768e-08 - accuracy: 0.0987
Epoch 9/10
1875/1875 [==============================] - 3s 2ms/step - loss: 1.1768e-08 - accuracy: 0.0987
Epoch 10/10
313/313 [=========================

## Stretch Goals: 

- Make MNIST a multiclass problem using cross entropy & soft-max
- Implement Cross Validation model evaluation on your MNIST implementation 
- Research different [Gradient Descent Based Optimizers](https://keras.io/optimizers/)
 - [Siraj Raval the evolution of gradient descent](https://www.youtube.com/watch?v=nhqo0u1a6fw)
- Build a housing price estimation model using a neural network. How does its accuracy compare with the regression models that we fit earlier on in class?